In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

/home/arh234/anaconda3/envs/dev-sanpi/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)

In [ ]:
UNIT = 'Adj'
# UNIT = 'Adv'
# UNIT = 'Bigr' 
# PAT_DIR = 'POSmirror'
# PAT_DIR = 'NEGmirror'
PAT_DIR = 'POSmirror'
# FRQ_FLOOR = 3
# FRQ_FLOOR = 10
FRQ_FLOOR = 20
# FRQ_FLOOR = 50
# FRQ_FLOOR = 100
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
        #  'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/POSmirror/readable/TrigAdj_frq-thrMIN-7.35f_min20x*` frequency data and initial associations...
ucs/trigger_eval/POSmirror/readable/TrigAdj_frq-thrMIN-7.35f_min20x.rsort-view_am-only.txt


# %% [markdown]

 Snippet of starting frequency data (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

23978  something  different
15511  something  simple
11890  something  wrong
10693  something  special
10204  something  important


# %% [markdown]

## 2. Run `confirm_basic_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    basic_ucs_path = readable.parent.parent.joinpath(
        readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
    print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

    basic_ucs_path = confirm_basic_ucs(
        basic_ucs_path,
        freq_floor=FRQ_FLOOR,
        contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/POSmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz

## Creating initial UCS table...

```
( cat /share/compling/projects/sanpi/results/freq_out/POSmirror/ucs_format/TrigAdj_frq-thrMIN-7.35f.tsv \ 
  | ucs-make-tables --types --threshold=20 /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz ) \ 
  && ucs-sort /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz BY f2- f1- INTO /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz
== Note ==
    N = total number of tokens/all counts summed
    V = total number of rows/number of unique l1+l2 combinations before filtering to 20+ tokens
+ time to make table → 00:00:00.560
```

Saving initial frequency table in readable .txt format...

```
ucs-print -o /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/readable/TrigAdj_frq-thrMIN-7.3

# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

        l1  l2                     f     f2      f1        N  
----------  -----------------  -----  -----  ------  -------  
        or  different           1615  34503  326614  1472077  
 something  different          23978  34503  318343  1472077  
       all  different           3628  34503  249279  1472077  
      some  different            472  34503  114929  1472077  
     often  different           1552  34503   92233  1472077  


# %% [markdown]

## 3. Run `associate_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-POSmirror_TrigAdj_frq-thrMIN-7-35f_min20x.2024-05-25_2219.log
...
```

+ time elapsed → 00:00:15.688
# Manipulating TrigAdj_frq-thrMIN-7-35f_min20x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 22:19:46 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/tmp/tmp_POSmirror-20240525-221946.TrigAdj_frq-thrMIN-7-35f_min20x)
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz

# Frequency signatures computed by the ucs-make-tables tool for relational cooccurrences.
# Sample size:  N = 1472077 tokens,  V = 61860 pair types.
# A frequency thre

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

        l1  l2                     f           E11  am.log.likelihood  am.odds.ratio.disc   am.p1.given2    am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1     f2        N  
----------  -----------------  -----  ------------  -----------------  ------------------  -------------  --------------  -------------------  -------------------  ------  -----  -------  
      many  bookable              32     1.2166850        187.5161378        2.1797313871   0.8101081490   0.00067471915         0.8421052632        6.7892984e-04   47133     38  1472077  
       all  gloom                 38     6.6041933        126.0396867        2.1001036563   0.8050420138   0.00015162184         0.9743589744        1.5243964e-04  249279     39  1472077  
everything  peachy                82     2.6843705        500.1713216        2.2095136999   0.7932101792   0.00206254391         0.8200000000        2.0751088e-03   39516    100  1472077  
UCS table text converted & saved as /share/compling/pro

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,many,bookable,32,1.22,187.52,2.18,0.81,0.00,0.84,0.00,47133,38,1472077
1,all,gloom,38,6.60,126.04,2.10,0.81,0.00,0.97,0.00,249279,39,1472077
2,everything,peachy,82,2.68,500.17,2.21,0.79,0.00,0.82,0.00,39516,100,1472077
3,something,amiss,537,116.78,"1,609.65",2.75,0.78,0.00,0.99,0.00,318343,540,1472077
4,or,least,104,23.07,313.20,2.87,0.78,0.00,1.00,0.00,326614,104,1472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8392,something,willing,28,843.18,"-1,642.69",-1.58,-0.21,-0.00,0.01,0.00,318343,3899,1472077
8393,something,excited,66,"1,906.93","-3,703.36",-1.56,-0.21,-0.01,0.01,0.00,318343,8818,1472077
8394,or,special,173,"2,986.41","-5,368.50",-1.34,-0.21,-0.01,0.01,0.00,326614,13460,1472077
8395,something,young,30,"1,425.76","-2,914.72",-1.78,-0.21,-0.01,0.00,0.00,318343,6593,1472077


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
many~bookable,many,bookable,32,1.22,187.52,2.18,0.81,0.00,0.84,0.00,47133,38,1472077
all~gloom,all,gloom,38,6.60,126.04,2.10,0.81,0.00,0.97,0.00,249279,39,1472077
everything~peachy,everything,peachy,82,2.68,500.17,2.21,0.79,0.00,0.82,0.00,39516,100,1472077
something~amiss,something,amiss,537,116.78,"1,609.65",2.75,0.78,0.00,0.99,0.00,318343,540,1472077
or~least,or,least,104,23.07,313.20,2.87,0.78,0.00,1.00,0.00,326614,104,1472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
something~willing,something,willing,28,843.18,"-1,642.69",-1.58,-0.21,-0.00,0.01,0.00,318343,3899,1472077
something~excited,something,excited,66,"1,906.93","-3,703.36",-1.56,-0.21,-0.01,0.01,0.00,318343,8818,1472077
or~special,or,special,173,"2,986.41","-5,368.50",-1.34,-0.21,-0.01,0.01,0.00,326614,13460,1472077


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key               |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:------------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| many~bookable     |      5.44 |                 1.42 |         0.00 |         0.81 |             0.81 |             0.00 |          30.78 |               0.96 |
| all~gloom         |      5.09 |                 0.76 |         0.00 |         0.81 |             0.81 |             0.00 |          31.40 |               0.83 |
| everything~peachy |      8.76 |                 1.48 |         0.00 |         0.79 |             0.79 |             0.00 |          79.32 |               0.97 |
| something~amiss   |     18.13 |                 0.66 |         0.00 |         0.78 |             0.78 |             0.00 |         420.22 |   

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/POSmirror/extra/TrigAdj_frq-thrMIN-7.35f_min20x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        # 'very', 
        # 'even', 
        # 'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        # 'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['different',
 'familiar',
 'likely',
 'similar',
 'larger',
 'unable',
 'akin',
 'unaware',
 'unrelated',
 'brief']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items():
        try: 
            polar_ex = exdf.loc[polarity, :]
        except KeyError: 
            continue
        else: 
                
            for topw in polar_ex.reset_index().l2.head(2).squeeze(): 
                if topw not in prone[UNIT]: 
                    prone[UNIT].append(topw)
            prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC        quant    f  unexp_f  dP1  dP2  \
polarity l1         l2                                                    
positive everything peachy    5.35    universal   82    79.32 0.79 0.00   
         something  amiss     4.86  existential  537   420.22 0.78 0.00   
         someone    senior    4.84  existential  112   107.95 0.54 0.00   
         or         third     4.83  existential  437   338.49 0.76 0.00   
         everyone   welcoming 4.79    universal  502   478.15 0.44 0.02   

                               dP1_simple  dP2_simple       G2        N  \
polarity l1         l2                                                    
positive everything peachy           0.82        0.00   500.17  1472077   
         something  amiss            0.99        0.00 1,609.65  1472077   
         someone    senior           0.56        0.00   602.64  1472077   
         or         third            0.98        0.00 1,247.91  1472077   
         everyone   welcoming        0.46        0.02 2,367.69  1472077   

                                   f1    f2  exp_f  
polarity l1         l2                              
positive everything peachy      39516   100   2.68  
         something  amiss      318343   540 116.78  
         someone    senior      29662   201   4.05  
         or         third      326614   444  98.51  
         everyone   welcoming   32327  1086  23.85

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1             quant    f  unexp_f  LRC  dP2  \
polarity l1         l2                                                        
positive many       bookable 0.81  hedged_universal   32    30.78 3.89 0.00   
         all        gloom    0.81         universal   38    31.40 0.00 0.00   
         everything peachy   0.79         universal   82    79.32 5.35 0.00   
         something  amiss    0.78       existential  537   420.22 4.86 0.00   
         or         least    0.78       existential  104    80.93 0.73 0.00   

                              dP1_simple  dP2_simple       G2        N  \
polarity l1         l2                                                   
positive many       bookable        0.84        0.00   187.52  1472077   
         all        gloom           0.97        0.00   126.04  1472077   
         everything peachy          0.82        0.00   500.17  1472077   
         something  amiss           0.99        0.00 1,609.65  1472077   
         or         least           1.00        0.00   313.20  1472077   

                                  f1   f2  exp_f  
polarity l1         l2                            
positive many       bookable   47133   38   1.22  
         all        gloom     249279   39   6.60  
         everything peachy     39516  100   2.68  
         something  amiss     318343  540 116.78  
         or         least     326614  104  23.07

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple        quant    f  unexp_f  LRC  dP1  dP2  \
polarity l1 l2                                                               
positive or least           1.00  existential  104    80.93 0.73 0.78 0.00   
            thereof         1.00  existential   71    55.25 0.17 0.78 0.00   
            upstart         1.00  existential   70    54.47 0.14 0.78 0.00   
            seventh         1.00  existential   20    15.56 0.00 0.78 0.00   
            mildewed        1.00  existential   20    15.56 0.00 0.78 0.00   

                      dP2_simple     G2        N      f1   f2  exp_f  
polarity l1 l2                                                        
positive or least           0.00 313.20  1472077  326614  104  23.07  
            thereof         0.00 213.81  1472077  326614   71  15.75  
            upstart         0.00 210.80  1472077  326614   70  15.53  
            seventh         0.00  60.23  1472077  326614   20   4.44  
            mildewed        0.00  60.23  1472077  326614   20   4.44

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2        quant      f   unexp_f  LRC  dP1  \
polarity l1         l2                                                       
positive everybody  excited   0.08    universal    665    617.29 3.61 0.07   
         something  different 0.07  existential  23978 16,516.58 2.96 0.49   
         everything right     0.06    universal   2739  2,405.63 3.20 0.20   
         everyone   excited   0.06    universal   2056  1,862.36 3.57 0.21   
         everybody  happy     0.06    universal    502    442.83 2.80 0.04   

                               dP1_simple  dP2_simple        G2        N  \
polarity l1         l2                                                     
positive everybody  excited          0.08        0.08  2,363.65  1472077   
         something  different        0.69        0.08 37,254.55  1472077   
         everything right            0.22        0.07  7,394.99  1472077   
         everyone   excited          0.23        0.06  6,537.08  1472077   
         everybody  happy            0.05        0.06  1,304.83  1472077   

                                   f1     f2    exp_f  
polarity l1         l2                                 
positive everybody  excited      7965   8818    47.71  
         something  different  318343  34503 7,461.42  
         everything right       39516  12419   333.37  
         everyone   excited     32327   8818   193.64  
         everybody  happy        7965  10935    59.17

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple        quant      f   unexp_f  LRC  \
polarity l1         l2                                                         
positive everybody  excited          0.08    universal    665    617.29 3.61   
         something  different        0.08  existential  23978 16,516.58 2.96   
         everything right            0.07    universal   2739  2,405.63 3.20   
         everyone   excited          0.06    universal   2056  1,862.36 3.57   
         everybody  happy            0.06    universal    502    442.83 2.80   

                               dP1  dP2  dP1_simple        G2        N  \
polarity l1         l2                                                   
positive everybody  excited   0.07 0.08        0.08  2,363.65  1472077   
         something  different 0.49 0.07        0.69 37,254.55  1472077   
         everything right     0.20 0.06        0.22  7,394.99  1472077   
         everyone   excited   0.21 0.06        0.23  6,537.08  1472077   
         everybody  happy     0.04 0.06        0.05  1,304.83  1472077   

                                   f1     f2    exp_f  
polarity l1         l2                                 
positive everybody  excited      7965   8818    47.71  
         something  different  318343  34503 7,461.42  
         everything right       39516  12419   333.37  
         everyone   excited     32327   8818   193.64  
         everybody  happy        7965  10935    59.17

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2        quant      f   unexp_f  LRC  \
polarity l1        l2                                                       
positive something different 37,254.55  existential  23978 16,516.58 2.96   
                   simple    23,325.76  existential  15511 10,625.82 2.88   
                   wrong     21,543.11  existential  11890  8,528.98 3.42   
                   special   20,667.13  existential  10693  7,782.22 3.64   
         all       familiar  14,386.08    universal   8180  6,030.76 3.01   

                              dP1  dP2  dP1_simple  dP2_simple        N  \
polarity l1        l2                                                     
positive something different 0.49 0.07        0.69        0.08  1472077   
                   simple    0.48 0.04        0.69        0.05  1472077   
                   wrong     0.55 0.03        0.77        0.04  1472077   
                   special   0.58 0.03        0.79        0.03  1472077   
         all       familiar  0.48 0.03        0.64        0.03  1472077   

                                  f1     f2    exp_f  
polarity l1        l2                                 
positive something different  318343  34503 7,461.42  
                   simple     318343  22590 4,885.18  
                   wrong      318343  15542 3,361.02  
                   special    318343  13460 2,910.78  
         all       familiar   249279  12692 2,149.24

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index:int=0):
    try: 
        example = prone_list[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(regex=f'~{example}$', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['different',
 'familiar',
 'likely',
 'similar',
 'larger',
 'unable',
 'akin',
 'unaware',
 'unrelated',
 'brief',
 'peachy',
 'amiss',
 'bookable',
 'gloom',
 'least',
 'thereof',
 'excited',
 'simple']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~different,positive,existential,23978,"16,516.58",2.96,0.49,0.07,0.69,0.08,"37,254.55",1472077,318343,34503,"7,461.42",something,different
everyone~different,positive,universal,201,-556.69,-1.39,-0.02,-0.02,0.01,0.01,-599.05,1472077,32327,34503,757.69,everyone,different
sometimes~different,positive,existential,505,"-1,246.01",-1.50,-0.04,-0.02,0.01,0.01,"-1,305.30",1472077,74707,34503,"1,751.01",sometimes,different
many~different,positive,hedged_universal,195,-909.72,-1.99,-0.03,-0.02,0.01,0.00,"-1,186.06",1472077,47133,34503,"1,104.72",many,different
always~different,positive,universal,375,"-1,551.63",-2.03,-0.05,-0.02,0.01,0.00,"-1,980.14",1472077,82200,34503,"1,926.63",always,different
some~different,positive,existential,472,"-2,221.74",-2.25,-0.07,-0.02,0.01,0.00,"-2,998.42",1472077,114929,34503,"2,693.74",some,different
or~different,positive,existential,1615,"-6,040.28",-2.34,-0.18,-0.02,0.05,0.00,"-8,468.00",1472077,326614,34503,"7,655.28",or,different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> familiar <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~familiar,positive,universal,8180,"6,030.76",3.01,0.48,0.03,0.64,0.03,"14,386.08",1472077,249279,12692,"2,149.24",all,familiar
something~familiar,positive,existential,1189,"-1,555.70",-1.18,-0.12,-0.01,0.09,0.00,"-1,363.36",1472077,318343,12692,"2,744.70",something,familiar
often~familiar,positive,hedged_universal,161,-634.22,-1.77,-0.05,-0.01,0.01,0.00,-792.04,1472077,92233,12692,795.22,often,familiar
sometimes~familiar,positive,existential,70,-574.11,-2.34,-0.05,-0.01,0.01,0.00,-869.12,1472077,74707,12692,644.11,sometimes,familiar
or~familiar,positive,existential,512,"-2,304.01",-2.41,-0.18,-0.01,0.04,0.00,"-3,383.39",1472077,326614,12692,"2,816.01",or,familiar
always~familiar,positive,universal,44,-664.71,-2.92,-0.05,-0.01,0.00,0.00,"-1,126.79",1472077,82200,12692,708.71,always,familiar


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> likely <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~likely,positive,existential,7355,"4,644.82",2.27,0.38,0.02,0.60,0.02,"8,252.07",1472077,326614,12215,"2,710.18",or,likely
someone~likely,positive,existential,943,696.87,1.77,0.06,0.02,0.08,0.03,"1,197.92",1472077,29662,12215,246.13,someone,likely
somebody~likely,positive,existential,81,58.75,1.02,0.00,0.02,0.01,0.03,93.39,1472077,2682,12215,22.25,somebody,likely
everything~likely,positive,universal,55,-272.90,-1.56,-0.02,-0.01,0.00,0.00,-357.57,1472077,39516,12215,327.90,everything,likely
sometimes~likely,positive,existential,101,-518.90,-1.91,-0.04,-0.01,0.01,0.00,-697.99,1472077,74707,12215,619.90,sometimes,likely
something~likely,positive,existential,386,"-2,255.55",-2.68,-0.19,-0.01,0.03,0.00,"-3,540.84",1472077,318343,12215,"2,641.55",something,likely


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> similar <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
always~similar,positive,universal,77,-304.66,-1.49,-0.04,-0.00,0.01,0.00,-378.17,1472077,82200,6835,381.66,always,similar


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> larger <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~larger,positive,existential,1221,691.61,1.54,0.28,0.00,0.50,0.00,945.69,1472077,318343,2448,529.39,something,larger
all~larger,positive,universal,66,-348.54,-1.91,-0.14,-0.00,0.03,0.00,-511.71,1472077,249279,2448,414.54,all,larger


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> unable <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~unable,positive,existential,839,564.10,2.41,0.46,0.00,0.68,0.00,"1,169.78",1472077,326614,1239,274.90,or,unable
many~unable,positive,hedged_universal,149,109.33,1.37,0.09,0.00,0.12,0.00,186.24,1472077,47133,1239,39.67,many,unable
all~unable,positive,universal,24,-185.81,-1.77,-0.15,-0.00,0.02,0.00,-299.41,1472077,249279,1239,209.81,all,unable


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> akin <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~akin,positive,existential,935,699.72,3.79,0.64,0.00,0.86,0.00,"2,056.39",1472077,318343,1088,235.28,something,akin
or~akin,positive,existential,65,-176.40,-1.18,-0.16,-0.00,0.06,0.00,-216.79,1472077,326614,1088,241.40,or,akin


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> unaware <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
many~unaware,positive,hedged_universal,290,259.01,3.15,0.27,0.01,0.30,0.01,859.69,1472077,47133,968,30.99,many,unaware


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> unrelated <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~unrelated,positive,existential,573,376.64,2.10,0.42,0.00,0.63,0.00,723.06,1472077,318343,908,196.36,something,unrelated
all~unrelated,positive,universal,25,-128.76,-1.28,-0.14,-0.00,0.03,0.00,-187.60,1472077,249279,908,153.76,all,unrelated


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> brief <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~brief,positive,universal,304,200.87,1.66,0.33,0.00,0.50,0.00,348.84,1472077,249279,609,103.13,all,brief
something~brief,positive,existential,25,-106.70,-1.11,-0.18,-0.00,0.04,0.00,-152.61,1472077,318343,609,131.70,something,brief


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['important', 'sure', 'late', 'frustrating', 'evident']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> important <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everybody~important,positive,universal,28,-129.55,-1.04,-0.00,-0.02,0.00,0.00,-165.09,1472077,7965,29118,157.55,everybody,important
some~important,positive,existential,777,"-1,496.32",-1.35,-0.05,-0.01,0.03,0.01,"-1,427.70",1472077,114929,29118,"2,273.32",some,important
many~important,positive,hedged_universal,182,-750.30,-1.82,-0.03,-0.02,0.01,0.00,-938.27,1472077,47133,29118,932.30,many,important
everyone~important,positive,universal,95,-544.44,-1.98,-0.02,-0.02,0.00,0.00,-746.45,1472077,32327,29118,639.44,everyone,important


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> sure <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~sure,positive,existential,1773,887.51,1.24,0.22,0.00,0.44,0.01,972.01,1472077,326614,3991,885.49,or,sure
sometimes~sure,positive,existential,530,327.46,1.16,0.08,0.00,0.13,0.01,395.41,1472077,74707,3991,202.54,sometimes,sure
all~sure,positive,universal,247,-428.83,-1.12,-0.11,-0.00,0.06,0.00,-414.53,1472077,249279,3991,675.83,all,sure
something~sure,positive,existential,59,-804.07,-3.19,-0.20,-0.00,0.01,0.00,"-1,485.11",1472077,318343,3991,863.07,something,sure


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> late <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
often~late,positive,hedged_universal,683,542.34,2.36,0.24,0.01,0.30,0.01,"1,230.83",1472077,92233,2245,140.66,often,late
some~late,positive,existential,31,-144.27,-1.20,-0.06,-0.00,0.01,0.00,-191.17,1472077,114929,2245,175.27,some,late


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> frustrating <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
sometimes~frustrating,positive,existential,268,202.18,1.75,0.16,0.00,0.21,0.00,383.97,1472077,74707,1297,65.82,sometimes,frustrating
all~frustrating,positive,universal,518,298.37,1.27,0.23,0.00,0.40,0.00,384.14,1472077,249279,1297,219.63,all,frustrating
something~frustrating,positive,existential,66,-214.48,-1.39,-0.17,-0.00,0.05,0.00,-280.53,1472077,318343,1297,280.48,something,frustrating


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> evident <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~evident,positive,universal,600,427.44,2.32,0.42,0.00,0.59,0.00,907.08,1472077,249279,1019,172.56,all,evident
or~evident,positive,existential,41,-185.09,-1.53,-0.18,-0.00,0.04,0.00,-270.50,1472077,326614,1019,226.09,or,evident


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, [r'exactly\w*'] if UNIT!='Adj' else [r'un\w+'])
exdf

>> un\w+ <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
many~uninsured,positive,hedged_universal,43,40.89,3.82,0.62,0.00,0.65,0.00,212.16,1472077,47133,66,2.11,many,uninsured
many~undecided,positive,hedged_universal,60,55.68,3.26,0.41,0.00,0.44,0.00,232.44,1472077,47133,135,4.32,many,undecided
many~unaware,positive,hedged_universal,290,259.01,3.15,0.27,0.01,0.30,0.01,859.69,1472077,47133,968,30.99,many,unaware
many~unaccounted,positive,hedged_universal,40,37.15,2.98,0.42,0.00,0.45,0.00,156.07,1472077,47133,89,2.85,many,unaccounted
something~unique,positive,existential,4064,"2,817.29",2.89,0.49,0.01,0.70,0.01,"6,313.09",1472077,318343,5765,"1,246.71",something,unique
something~unheard,positive,existential,352,253.82,2.77,0.56,0.00,0.78,0.00,644.26,1472077,318343,454,98.18,something,unheard
something~unexpected,positive,existential,1234,859.02,2.75,0.50,0.00,0.71,0.00,"1,942.77",1472077,318343,1734,374.98,something,unexpected
many~unemployed,positive,hedged_universal,71,63.60,2.64,0.28,0.00,0.31,0.00,214.15,1472077,47133,231,7.40,many,unemployed
or~unavailable,positive,existential,433,302.54,2.57,0.51,0.00,0.74,0.00,703.71,1472077,326614,588,130.46,or,unavailable
